# purpose

- this script helps you efficiently create new GitHub issues with customized properties, including name, description, and labels


# structure

- **fetch labels**
  - retrieve the list of available labels along with their IDs
  - save this information to a csv file for easy reference when creating new issues
- **fetch repositories**
  - retrieve the list of available repositories and their corresponding IDs
  - save this data to a csv file to streamline the process of assigning repositories to new issues
- **prepare issue details**
  - construct a json structure to define the new issue
  - customize the issue by adding the desired name, description, and other attributes
  - ensure to use the repository_id and label_ids from the csv files generated in the previous steps
- **create issue**
  - use the prepared json structure to create a new issue
  - receive a response containing the details of the newly created issue

# setup


In [ ]:
# fetch github token, python packages, queries and other parameters

%run ../../100-set_parameters/100-set_parameters.ipynb # magic commands

# set parameters

- set the parameters that will be used by the rest of the script


In [ ]:
# query parameters
# github does not allow fetching more than 100 records

fetch_issue_parameters = {
    "repository_name": "UPDATE_ME",
    "owner_name": "UPDATE_ME",
    "number_of_issues": 100,
    "number_of_labels": 100,
    "number_of_repositories": 100,
    "number_of_user_ids": 100,
    "login_user": "UPDATE_ME"
  }

# fetch labels


In [ ]:
# labels dictionary

labels_dictionary = global_query(fetch_100_labels, fetch_issue_parameters)

In [ ]:
# list of labels

## first create a list of labels
list_of_labels = labels_dictionary['data']['repository']['labels']['nodes']
labels_filename = '260-list_of_labels.csv'
labels_fieldnames = ['id', 'name']

# now dump it into a csv file
with open(labels_filename, 'w', newline='') as labels_csv_file:
    writer = csv.DictWriter(labels_csv_file, fieldnames=labels_fieldnames)
    writer.writeheader()
    writer.writerows(list_of_labels)

# fetch repositories


In [ ]:
# repository dictionary

repository_dictionary = global_query(fetch_100_repositories, fetch_issue_parameters)

In [ ]:
# list of repositories

# fetch list of repositories
list_of_repositories = repository_dictionary['data']['user']['repositories']['edges']
repositories_filename = '260-list_of_repositories.csv'
repositories_fieldnames = ['id', 'name']

# now dump it into a csv
with open(repositories_filename, 'w', newline='') as repositories_csv_file:
    writer = csv.DictWriter(repositories_csv_file, fieldnames=repositories_fieldnames)
    writer.writeheader()
    for row in list_of_repositories:
        writer.writerow({'id': row['node']['id'], 'name': row['node']['name']})

# fetch user ids


In [ ]:
# user id dictionary

user_id_dictionary = global_query(fetch_100_user_ids, fetch_issue_parameters)

In [ ]:
# list of user ids

# fetch list of user ids
list_of_user_ids = user_id_dictionary['data']['repository']['collaborators']['edges']
user_ids_filename = '260-list_of_user_ids.csv'
user_ids_fieldnames = ['id', 'name']

# dump it into a csv file
with open(user_ids_filename, 'w', newline='') as user_ids_csv_file:
    writer = csv.DictWriter(user_ids_csv_file, fieldnames=user_ids_fieldnames)
    writer.writeheader()
    for row in list_of_user_ids:
        writer.writerow({'id': row['node']['id'], 'name': row['node']['name']})

In [ ]:
# new user id

# github node ids are being migrated
# so in the previous query when you ask for all assigneeIDs, github gives you values for each user
# one value is the legacy_user_id and another value is the new_user_id
# use the new_user_id when creating issues in the next steps
# more details here - https://docs.github.com/en/graphql/guides/migrating-graphql-global-node-ids

# create single issue


In [ ]:
# specify issue details

# in the previous steps, we have fetched all the details that are needed to create the issue
# we fetched earlier the label ids to be passed as a list "labelIds" below
# then we also fetched the id for all our repositories - pass it as "repositoryId"
# "assigneeIds" has been fetched as user id in the previous step

create_issue_with_these_details = {
  "query_input": {
    "repositoryId": "R_kgDOHQ63-g",
    "title": "10 test issue",
    "body": "10 test issue description",
    "labelIds": ["LA_kwDOHQ63-s7ztW_L", "LA_kwDOHQ63-s7ztYm7", "LA_kwDOHQ63-s7ztYr4"],
    "assigneeIds": "U_kgDOAZEaHQ"
  }
  }

In [ ]:
# create issue mutation

create_issue_mutation = """
mutation ($query_input: CreateIssueInput!) {
  createIssue(input: $query_input) {
    issue {
      id
      number
      title
      body
      state
      labels(first: 100) {
        edges {
          node {
            id
          }
        }
      }
      assignees(first: 10) {
        edges {
          node {
            id
            name
          }
        }
      }
    }
  }
}
"""

In [ ]:
# execute mutation

issue_created = global_query(create_issue_mutation, create_issue_with_these_details)

In [ ]:
# print output

print(json.dumps(issue_created, indent=4))
